In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/cmpe255-orion/car-price-prediction/main/listing.csv')
del df['Unnamed: 0']
print(df.shape)
df.head(5)

(9936, 20)


,year,make,model,sub_model,city,state,mileage,price,style,exterior_color,interior_color,engine,drive_type,fuel_type,transmission,accidents,title,owners,use_type,cpo
0,2021,Chevrolet,Suburban,LT RWD,San Antonio,TX,"8,404","$67,994",SUV,Satin Steel Metallic,Jet Black,5.3L V-8 Gas,RWD,Gas,Automatic,0,Clean,1,Personal Use,True
1,2017,Cadillac,CTS,Premium Luxury 3.6 AWD,Phoenix,AZ,"29,342","$37,000",Sedan,Black Raven,Jet Black With Morello Red Accents,3.6L V-6 Gas,AWD,Gas,Automatic,0,Clean,1,Personal Use,True
2,2019,Kia,Forte,LXS IVT,Bensalem,PA,"17,593","$22,395",Sedan,Clear White,Black,2.0L Inline-4 Gas,FWD,Gas,Automatic,0,Clean,2,Personal Use,True
3,2018,Ford,F-150,XLT SuperCrew 5.5' Box 4WD,Hudson,WI,"195,205","$20,995",Pickup Truck,Shadow Black,Medium Earth Gray,2.7L V-6 Gas Turbocharged,4WD,Gas,Automatic,0,Clean,1,Personal Use,True
4,2018,Ford,F-150,XLT SuperCrew 5.5' Box 4WD,Pikeville,KY,"47,922","$32,988",Pickup Truck,Black,Black,3.3L V-6 Gas,4WD,Gas,Automatic,0,Clean,4,Rental Use,True


This dataset scraped from TrueCar has 9936 car listings with 20 columns. The following are the column descriptions:
<pre>
1. year:            Model Year
2. make:            Manufacturer
3. model:           Vehicle Model
4. sub_model:       Vehicle Model variant
5. city:            City where the car is located
6. state:           State where the car is located
7. mileage:         Car's current odometer reading
8. price:           Car's selling price
9. style:           Style of the car
10. exterior_color: Car's exterior color
11. interior_color: Car's interior color
12. engine:         Car's engine specification
13. drive_type:     Drive Type
14. fuel_type:      Type of fuel
15. transmission:   Type of transmission
16. accidents:      Number of accidents
17. title:          Type of title(clean or salvaged, etc..)
18. owners:         Number of previous owners
19. use_type:       Type of usage of the car
20. cpo:            Certified Pre-Owned or Not
</pre>



**Check for unique values**

In [3]:
print(df['style'].unique())
print(df['drive_type'].unique())
print(df['fuel_type'].unique())
print(df['transmission'].unique())
print(df['accidents'].unique())
print(df['title'].unique())
print(df['owners'].unique())
print(df['use_type'].unique())
print(df['cpo'].unique())

['SUV' 'Sedan' 'Pickup Truck' 'Minivan' 'Coupe' 'Hatchback' 'Cargo Van'
 'Convertible' 'Passenger Van' 'Wagon' 'Chassis Cab Truck']
['RWD' 'AWD' 'FWD' '4WD']
['Gas' 'Hybrid' 'Diesel' 'Electric' 'Plug-In Hybrid' 'Hydrogen']
['Automatic' 'Manual']
[0 1 2 3 5 4]
['Clean' 'Salvage History' 'Theft History' 'Lemon History']
[ 1  2  4  3  0  5  7  6  8  9 10]
['Personal Use' 'Rental Use' 'Mixed Use' 'Fleet Use']
[ True]


Here we find that cpo has only one value 'True'. Probably that field has wrong data. Hence it can be removed.

In [4]:
del df['cpo']
df.shape

(9936, 19)

Get rid of cars that are lemons.

In [5]:
df = df[df['title'] != 'Lemon History']
df.shape

(9932, 19)

**Check for missing values**

In [6]:
df.isna().sum()

year              0
make              0
model             0
sub_model         0
city              0
state             0
mileage           0
price             0
style             0
exterior_color    0
interior_color    0
engine            0
drive_type        0
fuel_type         0
transmission      0
accidents         0
title             0
owners            0
use_type          0
dtype: int64

We find that there are no missing values

**Check the data type of the columns**

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9932 entries, 0 to 9935
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            9932 non-null   int64 
 1   make            9932 non-null   object
 2   model           9932 non-null   object
 3   sub_model       9932 non-null   object
 4   city            9932 non-null   object
 5   state           9932 non-null   object
 6   mileage         9932 non-null   object
 7   price           9932 non-null   object
 8   style           9932 non-null   object
 9   exterior_color  9932 non-null   object
 10  interior_color  9932 non-null   object
 11  engine          9932 non-null   object
 12  drive_type      9932 non-null   object
 13  fuel_type       9932 non-null   object
 14  transmission    9932 non-null   object
 15  accidents       9932 non-null   int64 
 16  title           9932 non-null   object
 17  owners          9932 non-null   int64 
 18  use_type

We could conver the fields mileage to numeric and remove the $ sign and comma from the price.

In [8]:
df['mileage'] = df['mileage'].apply(lambda x: ''.join(x.split(',')))
df['mileage'] = df['mileage'].astype(int)
df['mileage'].head()

0      8404
1     29342
2     17593
3    195205
4     47922
Name: mileage, dtype: int64

In [9]:
df['price'] = df['price'].str.replace(r'$', '')
df['price'] = df['price'].str.replace(r',', '')
df['price'].head()

0    67994
1    37000
2    22395
3    20995
4    32988
Name: price, dtype: object

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9932 entries, 0 to 9935
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year            9932 non-null   int64 
 1   make            9932 non-null   object
 2   model           9932 non-null   object
 3   sub_model       9932 non-null   object
 4   city            9932 non-null   object
 5   state           9932 non-null   object
 6   mileage         9932 non-null   int64 
 7   price           9932 non-null   object
 8   style           9932 non-null   object
 9   exterior_color  9932 non-null   object
 10  interior_color  9932 non-null   object
 11  engine          9932 non-null   object
 12  drive_type      9932 non-null   object
 13  fuel_type       9932 non-null   object
 14  transmission    9932 non-null   object
 15  accidents       9932 non-null   int64 
 16  title           9932 non-null   object
 17  owners          9932 non-null   int64 
 18  use_type

**Check for duplicates and remove them**

In [11]:
print(df.duplicated().sum())
df.shape
df = df.drop_duplicates(keep='first')
df.shape

29


(9903, 19)

Apparently there were 29 duplicate records and we have removed them.

**Outlier Analysis**

Check the mileage and price for any discrepancy in values

In [12]:
display(df.sort_values(by='mileage',ascending=True).head(3))
display(df.sort_values(by='mileage',ascending=False).head(3))

,year,make,model,sub_model,city,state,mileage,price,style,exterior_color,interior_color,engine,drive_type,fuel_type,transmission,accidents,title,owners,use_type
3072,2020,Toyota,Corolla,LE CVT,Hardeeville,SC,9,19975,Sedan,Classic Silver Metallic,Light Gray,1.8L Inline-4 Gas,FWD,Gas,Automatic,0,Clean,1,Personal Use
3572,2019,BMW,7 Series,740i RWD,Houston,TX,12,56486,Sedan,Black Sapphire Metallic,Black,3.0L Inline-6 Gas Turbocharged,RWD,Gas,Automatic,0,Clean,1,Personal Use
404,2021,Mercedes-Benz,C-Class,C 300 Sedan RWD,Naples,FL,37,43850,Sedan,Polar White,Silk Beige,2.0L Inline-4 Gas Turbocharged,RWD,Gas,Automatic,0,Clean,1,Personal Use


,year,make,model,sub_model,city,state,mileage,price,style,exterior_color,interior_color,engine,drive_type,fuel_type,transmission,accidents,title,owners,use_type
46,2014,Ford,F-150,XLT SuperCab 8.0' Box 4WD,Cincinnati,OH,380309,10989,Pickup Truck,Tuxedo Black Metallic,Steel Gray,5.0L V-8 Gas,4WD,Gas,Automatic,1,Clean,2,Personal Use
9453,2014,Ram,3500 Chassis Cab,"4WD Crew Cab 172"" WB 60"" CA SLT",Murfreesboro,TN,362854,22495,Chassis Cab Truck,Black Clearcoat,Diesel Gray/Black,6.7L V-6 Diesel Turbocharged,4WD,Diesel,Automatic,0,Clean,1,Personal Use
3629,2018,Ram,3500,Tradesman Crew Cab 8' Box 4WD,Beaumont,TX,361367,33998,Pickup Truck,Bright White Clearcoat,Diesel Gray/Black,6.7L V-6 Diesel Turbocharged,4WD,Diesel,Automatic,0,Clean,1,Personal Use


By sorting the mileage data in both the ascending and descending order and looking for the minimum and maximum values, it does not look like there is any discrepancy in the data. A new car would have low miles and the lowest value which is 9 miles is nominal. In the same fashion the maximum value of 380309 miles for an old car is also very reasonable.

In [13]:
display(df.sort_values(by='price',ascending=False).head(5))

,year,make,model,sub_model,city,state,mileage,price,style,exterior_color,interior_color,engine,drive_type,fuel_type,transmission,accidents,title,owners,use_type
6919,2018,Toyota,Camry,XSE V6 Automatic,Lubbock,TX,49688,No Price Available,Sedan,Wind Chill Pearl,Cockpit Red,3.5L V-6 Gas,FWD,Gas,Automatic,0,Clean,1,Personal Use
9881,2018,Buick,Encore,Preferred II FWD,Gettysburg,PA,7343,No Price Available,SUV,Summit White,Shale,1.4L Inline-4 Gas Turbocharged,FWD,Gas,Automatic,0,Clean,1,Personal Use
3628,2020,Toyota,Corolla,LE CVT,Orlando,FL,51127,No Price Available,Sedan,Celestite Gray Metallic,Light Gray,1.8L Inline-4 Gas,FWD,Gas,Automatic,1,Clean,1,Personal Use
8058,2015,Ford,Fusion,SE FWD,Modesto,CA,156785,9999,Sedan,Oxford White,No Color,2.5L Inline-4 Gas,FWD,Gas,Automatic,0,Clean,2,Fleet Use
5801,2006,Nissan,Frontier,LE Crew Cab V6 4WD Auto,Orlando,FL,176790,9999,Pickup Truck,White,Unknown,4.0L V-6 Gas,4WD,Gas,Automatic,0,Clean,2,Personal Use


When sorting by price we find that there are some records which have the pricing information unavailable. Hence remove those records.

In [14]:
df = df[df['price'] != 'No Price Available']
df.shape

(9900, 19)

Convert the price to numeric and sort again

In [15]:
df['price'] = df['price'].astype(int)

In [16]:
display(df.sort_values(by='price',ascending=False).head(5))
display(df.sort_values(by='price',ascending=True).head(5))

,year,make,model,sub_model,city,state,mileage,price,style,exterior_color,interior_color,engine,drive_type,fuel_type,transmission,accidents,title,owners,use_type
2139,2011,Rolls-Royce,Ghost,RWD,Houston,TX,44818,114995,Sedan,English White,Tan,6.6L V-12 Gas Turbocharged,RWD,Gas,Automatic,0,Clean,4,Personal Use
7191,2017,Land,Rover Range Rover,V8 Supercharged SV Autobiography LWB,Newport Beach,CA,20087,107400,SUV,Scotia Grey,Black,5.0L V-8 Gas Supercharged,4WD,Gas,Automatic,0,Clean,1,Personal Use
5906,2012,Porsche,911,Turbo S Cabriolet,Lubbock,TX,45182,106969,Convertible,Black,Black,3.8L Flat-6 Gas Turbocharged,AWD,Gas,Automatic,0,Clean,3,Personal Use
3413,2012,Aston,Martin Virage,Coupe,Boerne,TX,8287,99888,Coupe,Morning Frost White,Unknown,5.9L V-12 Gas,RWD,Gas,Automatic,0,Clean,4,Personal Use
3108,2022,Mercedes-Benz,GLS,GLS 450 4MATIC,Newport Beach,CA,148,92893,SUV,Black,Black,3.0L V-6 Hybrid Turbocharged,AWD,Hybrid,Automatic,0,Clean,0,Personal Use


,year,make,model,sub_model,city,state,mileage,price,style,exterior_color,interior_color,engine,drive_type,fuel_type,transmission,accidents,title,owners,use_type
1179,1998,Pontiac,Bonneville,4dr Sedan SE,Tappahannock,VA,279213,2000,Sedan,Arctic White,Dark Pewter,3.8L V-6 Gas,FWD,Gas,Automatic,0,Clean,4,Rental Use
8308,2001,Buick,Regal,LS,Devils Lake,ND,195000,2000,Sedan,Sterling Silver Metallic,Medium Gray,3.8L V-6 Gas,FWD,Gas,Automatic,1,Clean,5,Mixed Use
624,2001,Hyundai,Accent,GS Hatchback Manual,Sandy,UT,148360,2200,Hatchback,Ebony Black,Unknown,1.6L Inline-4 Gas,FWD,Gas,Manual,0,Clean,5,Personal Use
4709,2002,Chevrolet,Suburban,LS 1500 4WD,Peabody,MA,289452,2495,SUV,Red,Unknown,5.3L V-8 Gas,4WD,Gas,Automatic,0,Clean,3,Personal Use
7580,1998,Oldsmobile,Cutlass,4dr Sedan GLS,Lebanon,IN,94585,2495,Sedan,Tan,Tan,3.1L V-6 Gas,FWD,Gas,Automatic,1,Clean,4,Personal Use


The lowest price of 2000 and the highest price of 114995 is well within the acceptable range for car prices.

In [17]:
df.shape

(9900, 19)

In [18]:
df.to_csv('listing-cleaned.csv', encoding = 'utf-8')